In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


# Create GRAPHRAG DB

In [3]:
from tools.micro_process import ProcessMicroData
from langchain_anthropic import ChatAnthropic
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o", model_provider="openai")
# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

gen_graphdb = ProcessMicroData(llm=llm)
graph, graph_documents = gen_graphdb.analyze_codebase(repo_path='../docs/notebooks/assignment-2-mcdonald-s/src')

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src\tools\micro_process.py:216: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  repo_tree_string = generate_repo_tree(repo_path)


Processing file: deduplication\bloom_filter.py
Processing file: deduplication\dedup.py
Processing file: deduplication\LSH.py
Processing file: deduplication\LSHForest.py
Processing file: deduplication\LSHImproved.py
Processing file: deduplication\__init__.py
Processing file: deduplication\__main__.py
Processing file: utils\use_cases.py
Processing file: utils\utils.py
Processing file: utils\visualizations.py
Processing file: utils\visualization_lsh.py
Converting to graph documents...
Adding graph documents to Neo4j...


# Visualize Entire Graph

In [4]:
a = gen_graphdb.create_visualization(graph_documents = graph_documents)

Graph saved as semantic_graph.html


# CREATE VECTOR DB

In [2]:
from tools.micro_text_answer import create_rag_system

db_setup, query_engine = create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="claude-3-5-sonnet-20240620",
)


# Cypher Query to generate initial Answer

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [4]:
from langchain_neo4j import Neo4jGraph
from tools.micro_cypher_chain import CypherGraphBuilder
from langchain_anthropic import ChatAnthropic




url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph_db = Neo4jGraph(
    url=url,
    username=username,
    password=password,
    enhanced_schema=True
)


# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o", model_provider="openai")


builder = CypherGraphBuilder(llm=llm, graph_db=graph_db)
cypher_graph = builder.create_cypher_graph()

done
done


In [5]:
from langchain_core.language_models import BaseChatModel
from langchain_neo4j import Neo4jGraph
from tools.micro_cypher_chain import CypherGraphBuilder

url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph_db = Neo4jGraph(
    url=url,
    username=username,
    password=password,
    enhanced_schema=True
    )

def retrieve_cypher_relationships(question: str):
    """
    Connects to a local Neo4j instance and uses a language model to generate a Cypher-based graph
    representation of relationships between entities (e.g., code modules).

    Returns:
        dict: A structured representation of nodes and relationships inferred from the graph.
    """

    # Create the Cypher graph
    builder = CypherGraphBuilder(llm=llm, graph_db=graph_db)
    # cypher_graph = builder.create_cypher_graph()
    cypher_graph = builder.create_cypher_graph()
    result = cypher_graph.invoke({"question": question})

    return result['answer']

In [3]:
retrieve_cypher_relationships("How is the Utils.Utils module related to the Deduplication.__Main__ module?")

done
done


TypeError: Can't instantiate abstract class BaseChatModel without an implementation for abstract methods '_generate', '_llm_type'

In [3]:
result = cypher_graph.invoke({"question": "How is the Utils.Utils module related to the Deduplication.__Main__ module?"})

In [5]:
result['answer']

'- The `Utils.Utils` module is imported in `Deduplication.__Main__`.\n- Functions from `Utils.Utils` such as `Read_Tsv`, `Minhash`, `Shingle`, `Clean_Document`, and `Majority_Vote` are used by `Deduplication.__Main__`.\n- The `Utils.Utils.Unionfind` class is used in the `Deduplication.__Main__` module through its association with `Collection_Deduplication`.\n- The `Utils.Utils` module imports and uses components like `Deduplication.Dedup.Baseline` which interact with `Deduplication.__Main__`.'

In [6]:
result['cypher_results']

[{'path': [{'id': 'Utils.Utils'},
   'CLASS',
   {'id': 'Utils.Utils.Unionfind'},
   'USED_IN',
   {'id': 'Utils.Use_Cases.Collection_Deduplication'},
   'DEFINES',
   {'id': 'Utils.Use_Cases'},
   'DEFINES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'IMPORTS',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'CLASS',
   {'id': 'Utils.Utils.Unionfind'},
   'USED_IN',
   {'id': 'Utils.Use_Cases.Collection_Deduplication'},
   'DEFINES',
   {'id': 'Utils.Use_Cases'},
   'DEFINES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'USES',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'CLASS',
   {'id': 'Utils.Utils.Unionfind'},
   'USED_IN',
   {'id': 'Utils.Use_Cases.Collection_Deduplication'},
   'IMPORTS',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'CLASS',
   {'id': 'Utils.Utils.Unionfind'},
   'USED_IN',
   {'id': 'Utils.Use_Cases.Collection_Deduplication'},
   'IMPORTS',
  

In [7]:
# graph_db.query("""
# MATCH path = (m1:Module {id: 'Utils.Utils'})-[*..5]-(m2:Module {id: 'Deduplication.__Main__'}) 
# WHERE NONE(n IN nodes(path) WHERE n:Package)
# RETURN path
# """)

In [13]:
# import os
# print(os.getcwd())
# os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
# print(os.getcwd())

# from tools.micro_tools import (retrieve_cypher_relationships)

# cypher_generator = retrieve_cypher_relationships("How is the Utils.Utils module related to the Deduplicatioin.__Main__ module?")

# Text Final Answer

In [9]:
questions = """
The Utils.Utils module is intricately related to the Deduplication.__Main__ module through several intermediate connections, reflecting a well-structured framework that enhances document deduplication capabilities. Here are the key points of this relationship:

1. **Function Utilization**: The Utils.Utils module utilizes the `Compute_Minhash_Signatures` function from Deduplication.Lsh.Lsh, which plays a crucial role in generating MinHash signatures for documents. This computational function is essential for the efficiency of the deduplication process, as it allows for quick similarity comparisons between documents based on their hashed representations.

2. **Class Interdependency**: The Deduplication.Lsh.Lsh module invokes the `Model` class from Deduplication.__Main__. This class serves as the backbone of the deduplication system, encapsulating core functionalities and configurations necessary for implementing various LSH-based methods effectively. The relationship emphasizes a modular design where the Lsh module extends the capabilities defined in the __Main__ module, facilitating code maintainability and scalability.

3. **Definition of Key Classes**: The Deduplication.__Main__ module independently defines both the `Model` class and the `__Main__` class. The `Model` class encompasses methods for executing the deduplication algorithms and managing the data structures involved, while the `__Main__` class is responsible for orchestrating the overall execution flow of the deduplication process. This design fosters a clear separation of concerns, allowing more straightforward debugging and enhancement in future iterations.

4. **Indirect Connections through Variants**: Additional relationships exist through other modules like `Lshimproved.Lsh` and `Lshforest.Lshforest`, which are specifically imported by Deduplication.__Main__. These imports signify that the main deduplication script leverages advancements in LSH techniques, such as improved processes for handling larger datasets and enhanced accuracy in identifying duplicate documents.

In summary, the Utils.Utils module is primarily connected to Deduplication.__Main__ through shared components and dependencies within the overall Deduplication system. These connections enhance the utility and efficiency of the deduplication process, particularly through the Lsh module and its associated classes and functions. The architecture aligns with best practices in software development by promoting modularity, enabling future enhancements like the implementation of new LSH-based algorithms without extensive modifications to the existing structure. This design not only streamlines the deduplication process but also positions the system for potential application in various scenarios, including plagiarism detection and content clustering.
"""
# Query the system
answer = query_engine.query(result['answer'])
print(answer)

Based on the provided sentence and context, I can add relevant details to make it more informative and useful:

The `Utils.Utils` module is a crucial component in the `Deduplication.__Main__` module, providing essential functionality for document processing and deduplication. Specifically:

1. The `Utils.Utils` module offers several key functions that are utilized in `Deduplication.__Main__`:

   - `Read_Tsv`: This function reads tab-separated values (TSV) files, likely used to load document datasets for deduplication.
   - `Minhash`: Implements the MinHash algorithm to create compact signatures of document shingles, essential for efficient similarity estimation between documents.
   - `Shingle`: Generates k-shingles from text, preserving some word order information for document comparison.
   - `Clean_Document`: Normalizes text by converting to lowercase and removing non-alphabetic characters, crucial for preprocessing before analysis.
   - `Majority_Vote`: Performs a majority vote on

# Visualization Final Answer

In [10]:
from tools.micro_visual_response import visualize_cypher_results
stats = visualize_cypher_results(result['cypher_results'], "code_relationships_graph.html")

Graph visualization saved as code_relationships_graph.html


# Agent

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


from tools.micro_text_answer import create_rag_system

db_setup, query_engine = create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="claude-3-5-sonnet-20240620",
)


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [2]:
from tools.micro_tools import (retrieve_cypher_relationships, visualize_relationships, generate_text_response)
from tools import micro_tools
from langgraph.prebuilt import create_react_agent

micro_tools.query_engine = query_engine

tools = [
    retrieve_cypher_relationships,
    visualize_relationships,
    generate_text_response,
]

system_prompt = f"""
You are a suupervisor that answers the questions given the following tools.
{tools}

You will use each tool only once and the guidelines of the tools are as follows:
- You will always use the retrieve_cypher_relationships tool to get an initial answer.
- You will always use the generate_text_response tool to get a final answer.
- Only use the visualize_relationships tool to visualize the relationships when the user asks for a visualization.

Focus on detailed implementation and execution. For high-level planning,
defer to the macro agent.

Always provide clear, actionable responses with specific details and implementation steps.
"""

from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o", model_provider="openai")

agent = create_react_agent(llm, tools=tools, prompt=system_prompt)

In [3]:
answer = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "How is the Utils.Utils module related to the Deduplication.__Main__ module? Can you give me a visualization as well?",
            }
        ],
        "goto": "supervisor",
    }
)

importing tools


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


initializing Neo4jGraph
Building Cypher graph
done
done
Invoking cypher graph
Visualizing relationships...
Results: {'answer': 'The `Utils.Utils` module is related to the `Deduplication.__Main__` module in multiple ways:\n\n- The `Utils.Utils` module, specifically `Utils.Utils.Unionfind`, is used in `Utils.Use_Cases.Collection_Deduplication` and either imports or uses `Deduplication.__Main__`.\n- The `Utils.Utils` module, through the `Read_Tsv` function, imports and/or uses the `Deduplication.__Main__` module.\n- The `Utils.Utils` module, via the `Minhash` function, uses `Deduplication.Lsh.Lsh.Compute_Minhash_Signatures` and interacts with `Deduplication.__Main__`.\n- The `Utils.Utils` module, specifically the `Shingle` function, is used in `Utils.Use_Cases.Nearest_Neighbor_Search` and interacts with `Deduplication.__Main__`.\n- The `Utils.Utils` module is imported in the `Deduplication.Lshimproved` module, which in turn imports `Deduplication.__Main__`.', 'steps': ['generate_cypher', 

In [5]:
print(answer['messages'][-1].content)

The `Utils.Utils` module is intricately related to the `Deduplication.__Main__` module, serving as a foundational toolkit with multiple significant interactions:

1. **UnionFind for Document Clustering**:
   - The `Utils.Utils.Unionfind` class is crucial for clustering documents based on candidate pairs identified by LSH processes in `Deduplication.__Main__`. It is used within `Utils.Use_Cases.Collection_Deduplication`.

2. **Data Input Processing**:
   - The `Read_Tsv` function from `Utils.Utils` reads TSV files and converts them into a format usable by `Deduplication.__Main__`, typically at the pipeline's start.

3. **MinHash Signature Generation**:
   - The `Minhash` function forms MinHash signatures crucial for similarity estimation, interacting with `Deduplication.Lsh.Lsh.Compute_Minhash_Signatures`, likely within the `compute_minhash_signatures` method invoked by `Deduplication.__Main__`.

4. **Document Preprocessing**:
   - The `Shingle` function generates k-shingles, vital for 

In [4]:
answer

{'messages': [HumanMessage(content='How is the Utils.Utils module related to the Deduplication.__Main__ module? Can you give me a visualization as well?', additional_kwargs={}, response_metadata={}, id='93292251-b958-4aaa-acda-f0956832283b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xumAZ62KiNB87z21VqqKcaU5', 'function': {'arguments': '{"question":"How is the Utils.Utils module related to the Deduplication.__Main__ module?"}', 'name': 'retrieve_cypher_relationships'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 731, 'total_tokens': 766, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_85bc8e3fae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-56e374d2-9391-4c

In [ ]:
print()